This notebook is an experiment to create an ANN model, especially **Feed-Forward Neural Network** for predicting the closing price of EURUSD based on daily closing price


In [1]:
#Necessary Imports
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import date



Though I mentioned in my paper, I will use the date from https://forexsb.com/historical-forex-data. I feel using this data, will be a time comsuming  becaue eveytime, I need to donwnload it and clean for the model. And python script does not work to download. So, it is best to use **yfinance** package for the time series forex data for EUR-USD.

In [2]:
#Download Dataset
TICKER = "EURUSD=X"
RANGE_FROM = "2015-01-01"
RANGE_TO = date.today().isoformat() #Data will be download till current day.

SAVE_DIRECTIRY = "forex_data"
FILE_NAME = "eurusd_daily.csv"

def download_data(ticker, start, end, savePath):
  print(f"Downloading Data for EUR-USD from {RANGE_FROM} to {RANGE_TO}")
  try:
    data = yf.download(ticker, start= start, end=end, progress= True)

    if data.empty:
      print(f"No data found for the {TICKER}")
    else:
      print("Successfully downloaded")
      os.makedirs(os.path.dirname(savePath), exist_ok=True)
      data.to_csv(savePath)
      print("Date saved")

  except Exception as e:
    print(f"An unexpected error: {e}")


full_save_path = os.path.join(SAVE_DIRECTIRY, FILE_NAME)
download_data(TICKER, RANGE_FROM, RANGE_TO, full_save_path)


/tmp/ipython-input-2-341652406.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start= start, end=end, progress= True)
[*********************100%***********************]  1 of 1 completed

Successfully downloaded
Date saved


In [37]:
#load csv and view from directory
if os.path.exists(full_save_path):
  forex_df = pd.read_csv(full_save_path, skiprows=3, header=None)
else:
  print("File not found")

#Cleaning Dataset. Remove every other column except Date and Close
forex_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
forex_df.columns = forex_df.columns.str.replace(' ','')
forex_df.drop(columns=['High', 'Low', 'Open', 'Volume'], inplace=True)


print(forex_df.tail())

            Date     Close
2736  2025-07-07  1.178078
2737  2025-07-08  1.173654
2738  2025-07-09  1.172457
2739  2025-07-10  1.173117
2740  2025-07-11  1.170275


Data Preparation for Feed-Forward neural Network

In [36]:
from sklearn.preprocessing import MinMaxScaler
closing_prices = forex_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(closing_prices)

In [7]:
SEQUENCE_LENGTH = 30
TRAIN_SPLIT = 0.7
VALIDATION_SPLIT = 0.15
TEST_SPLIT = 1 - TRAIN_SPLIT - VALIDATION_SPLIT

X, y = [], []
for i in range(len(scaled_prices) - SEQUENCE_LENGTH):
  X.append(scaled_prices[i:i+SEQUENCE_LENGTH])
  y.append(scaled_prices[i+SEQUENCE_LENGTH])

X = np.array(X)
y = np.array(y)

print(f"Created {len(X)} seq len {SEQUENCE_LENGTH}")

train_size = int(TRAIN_SPLIT * len(X))
val_size = int(VALIDATION_SPLIT * len(X))
test_size = len(X) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[-test_size:], y[-test_size:]

print(f"Train Size: {len(X_train)}")
print(f"Validation Size: {len(X_val)}")
print(f"Test Size: {len(X_test)}")

#Saving Datasets
SAVE_SETS_DIR = "prepared_sets"
os.makedirs(SAVE_SETS_DIR, exist_ok=True)
np.save(os.path.join(SAVE_SETS_DIR, "X_train.npy"), X_train)
np.save(os.path.join(SAVE_SETS_DIR, "y_train.npy"), y_train)
np.save(os.path.join(SAVE_SETS_DIR, "X_val.npy"), X_val)
np.save(os.path.join(SAVE_SETS_DIR, "y_val.npy"), y_val)
np.save(os.path.join(SAVE_SETS_DIR, "X_test.npy"), X_test)
np.save(os.path.join(SAVE_SETS_DIR, "y_test.npy"), y_test)


Created 2711 seq len 30
Train Size: 1897
Validation Size: 406
Test Size: 408


In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

#Creating Model
EPOCHS = 50
BATCH_SIZE = 32

X_train = np.load(os.path.join(SAVE_SETS_DIR, "X_train.npy"))
y_train = np.load(os.path.join(SAVE_SETS_DIR, "y_train.npy"))
X_val = np.load(os.path.join(SAVE_SETS_DIR, "X_val.npy"))
y_val = np.load(os.path.join(SAVE_SETS_DIR, "y_val.npy"))
X_test = np.load(os.path.join(SAVE_SETS_DIR, "X_test.npy"))
y_test = np.load(os.path.join(SAVE_SETS_DIR, "y_test.npy"))

MODEL = Sequential(
    [
        Input(shape=(SEQUENCE_LENGTH, 1)),
        Dense(64, activation='sigmoid'),
        Dense(1, activation ='linear')
    ]
)

MODEL.compile(optimizer='adam', loss='mse', metrics=['mae'])
MODEL.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 30, 64)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 30, 1)          │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

 Non-trainable params: 0 (0.00 B)